![https://i.ytimg.com/vi/aircAruvnKk/hqdefault.jpg](https://i.ytimg.com/vi/aircAruvnKk/hqdefault.jpg)

[Neural networks (3Blue1Brown) · Course](https://youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&si=Z4aX4QoYJJe9gOiw)

![https://i.ytimg.com/vi/l8pRSuU81PU/hqdefault.jpg](https://i.ytimg.com/vi/l8pRSuU81PU/hqdefault.jpg)

[Let's reproduce GPT-2 (124M)](https://youtu.be/l8pRSuU81PU)

![Transormer Mimarisi](transformer_mimari.png)

![Attention Blok Şeması](attention_blok_semasi.png)

In [1]:
import math  # Matematiksel fonksiyonlar ve sabitler için kullanılır.
from dataclasses import dataclass  # Veri sınıflarını basit ve anlaşılır bir şekilde tanımlamak için kullanılır.

import torch  # PyTorch kütüphanesi, makine öğrenmesi ve yapay zeka modelleri için kullanılır.
import torch.nn as nn  # PyTorch'un sinir ağı (neural network) modüllerini içerir.
from torch.nn import functional as F  # PyTorch'un sinir ağı işlevselliklerini içerir.

In [2]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # tüm başlıklar için key, query, value projeksiyonları, ancak toplu olarak
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # çıktı projeksiyonu
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # düzenleme
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        # aslında bir 'bias' değil, daha çok bir maske, ancak OpenAI/HF adlandırmasını takip ediyor
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))

    def forward(self, x, print_info=False):
        # x'in şekli [1, 8, 768]
        if print_info:
            print("B, T, C = x.size() = ", x.size())
        B, T, C = x.size()  # batch, sıra uzunluğu, gömme boyutu (n_embd)
                            # C, transformerdeki kanal sayısıdır, yani kanal sayısı gömme boyutuna eşittir.
        # tüm başlıklar için query, key, value hesaplayın ve başlığı ileriye taşıyın
        # nh "başlık sayısı"dır, hs "başlık boyutu"dur ve C (kanal sayısı) = nh * hs'dir
        # örneğin GPT-2 (124M) modelinde, n_head=12, hs=64, bu nedenle nh * hs=C=768 Kanal transformerde
        qkv = self.c_attn(x)
        if print_info:
            print("qkv.size() = ", qkv.size()) # qkv.size() =  torch.Size([1, 8, 2304])
        q, k, v = qkv.split(self.n_embd, dim=2)
        if print_info:
            print("q.size() = ", q.size()) # q.size() =  torch.Size([1, 8, 768])
            print("k.size() = ", k.size()) # k.size() =  torch.Size([1, 8, 768])
            print("v.size() = ", v.size()) # v.size() =  torch.Size([1, 8, 768])
        # q, k, v = [B, T, C] -> [B, T, 3, nh, hs] -> [B, nh, T, hs]
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        if print_info:
            print("q.size() = ", q.size()) # q.size() =  torch.Size([1, 12, 8, 64])
            print("k.size() = ", k.size()) # k.size() =  torch.Size([1, 12, 8, 64])
            print("v.size() = ", v.size()) # v.size() =  torch.Size([1, 12, 8, 64])
        # bu, tüm query ve key'ler için büyük (T,T) matrisini oluşturur
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1))) # (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
                                                                        # (1, 12, 8, 64) x (1, 12, 64, 8) -> (1, 12, 8, 8)
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf')) # matrisi maskelemek için üst üçgeni -inf ile doldur
                                                                        # modelin gelecekteki token'lara dikkat etmesini önlemek için
        att = F.softmax(att, dim=-1) # (B, nh, T, T) son boyutta softmax
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # tüm başlık çıktıları yan yana yeniden birleştirilir
        # çıktı projeksiyonu
        y = self.c_proj(y)
        return y

In [3]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)  # Tam bağlantılı katman, girişten çıkışa lineer dönüşüm
        self.gelu    = nn.GELU(approximate='tanh')  # Aktivasyon fonksiyonu olarak GELU (Gaussian Error Linear Unit)
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)  # Çıkış projeksiyonu, giriş boyutunu eski haline getirir

    def forward(self, x):
        x = self.c_fc(x)  # İlk tam bağlantılı katmandan geçiş
        x = self.gelu(x)  # GELU aktivasyon fonksiyonunu uygula
        x = self.c_proj(x)  # Çıkış projeksiyonu katmanından geçiş
        return x  # Çıktıyı döndür

In [4]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)  # Birinci Katman Normu
        self.attn = CausalSelfAttention(config)  # Nedensel Kendine Dikkat Katmanı
        self.ln_2 = nn.LayerNorm(config.n_embd)  # İkinci Katman Normu
        self.mlp = MLP(config)  # Çok Katmanlı Algılayıcı Katmanı

    def forward(self, x, print_info=False):
        ln_1_x = self.ln_1(x)  # İlk Katman Normunu uygula
        if print_info:
            print("LayerNorm ln_1_x", ln_1_x.shape, ln_1_x)  # İlk Katman Normunun çıktısını yazdır
        x = x + self.attn(ln_1_x)  # Dikkat katmanından gelen çıktıyı girişe ekle
        x = x + self.mlp(self.ln_2(x))  # İkinci Katman Normunu uygula ve MLP katmanından gelen çıktıyı ekle
        return x  # Çıktıyı döndür

In [5]:
""" @dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768 """

' @dataclass\nclass GPTConfig:\n    block_size: int = 1024\n    vocab_size: int = 50257\n    n_layer: int = 12\n    n_head: int = 12\n    n_embd: int = 768 '

Markdown'daki bazı ufak düzeltmeler yaparak metni yeniden düzenledim:

GPT modelinin parametre sayısını hesaplamak için aşağıdaki formüller kullanılır:

1. **Embedding katmanı**: \( vocab\_size \times n\_embd \)
2. **Transformer bloğu**: Her bir Transformer bloğunda toplam parametre sayısı şu şekildedir:
   - **LayerNorm**: İki LayerNorm katmanı vardır, her biri \( 2 \times n\_embd \)
   - **Dikkat (Q, K, V)**: \( 3 \times (n\_embd \times (n\_embd // n\_head) + n\_embd) \)
   - **Dikkat çıktı projeksiyonu**: \( n\_embd \times n\_embd + n\_embd \)
   - **MLP (ara katmanlar)**: \( 2 \times (n\_embd \times 4 \times n\_embd + 4 \times n\_embd) \)

Bu formülleri kullanarak parametre sayısını hesaplayalım.

### Hesaplamalar

#### Embedding Katmanı

\[
50257 \times 768 = 38,609,856
\]

#### Transformer Bloğu

Her bir Transformer bloğunda:

- **LayerNorm**: 
  \[
  2 \times 768 = 1,536
  \]

- **Dikkat (Q, K, V)**:
  \[
  3 \times (768 \times (768 // 12) + 768) = 3 \times (768 \times 64 + 768) = 3 \times (49,152 + 768) = 3 \times 49,920 = 149,760
  \]

- **Dikkat çıktı projeksiyonu**:
  \[
  768 \times 768 + 768 = 590,592 + 768 = 591,360
  \]

- **MLP**:
  \[
  2 \times (768 \times 4 \times 768 + 4 \times 768) = 2 \times (3,145,728 + 3,072) = 2 \times 3,148,800 = 6,297,600
  \]

Her bir Transformer bloğu için toplam:
\[
1,536 + 149,760 + 591,360 + 6,297,600 = 7,040,256
\]

#### Toplam Transformer Bloğu Parametreleri
\[
12 \times 7,040,256 = 84,483,072
\]

#### Toplam Parametreler
\[
38,609,856 (Embedding) + 84,483,072 (Transformer blokları) = 123,092,928
\]

Bu hesaplamalar, yaklaşık 124 milyon parametreye oldukça yakındır. Modelin toplam parametre sayısının 124M olduğu, verilen konfigürasyon değerlerine göre bu şekilde hesaplanır.

In [6]:
@dataclass
class GPTConfig:
    block_size: int = 64  # Blok boyutu
    vocab_size: int = 32  # Kelime hazinesi boyutu
    n_layer: int = 12     # Katman sayısı
    n_head: int = 12      # Başlık sayısı
    n_embd: int = 144     # Gömme boyutu

Bu konfigürasyon değerlerine göre modelin parametre sayısını yeniden hesaplayalım.

### Hesaplamalar

#### Embedding Katmanı

\[
32 \times 144 = 4,608
\]

#### Transformer Bloğu

Her bir Transformer bloğunda:

- **LayerNorm**: 
  \[
  2 \times 144 = 288
  \]

- **Dikkat (Q, K, V)**:
  \[
  3 \times (144 \times (144 // 12) + 144) = 3 \times (144 \times 12 + 144) = 3 \times (1,728 + 144) = 3 \times 1,872 = 5,616
  \]

- **Dikkat çıktı projeksiyonu**:
  \[
  144 \times 144 + 144 = 20,736 + 144 = 20,880
  \]

- **MLP**:
  \[
  2 \times (144 \times 4 \times 144 + 4 \times 144) = 2 \times (82,944 + 576) = 2 \times 83,520 = 167,040
  \]

Her bir Transformer bloğu için toplam:
\[
288 + 5,616 + 20,880 + 167,040 = 193,824
\]

#### Toplam Transformer Bloğu Parametreleri
\[
12 \times 193,824 = 2,325,888
\]

#### Toplam Parametreler
\[
4,608 (Embedding) + 2,325,888 (Transformer blokları) = 2,330,496
\]

Bu hesaplamalar, yaklaşık 2.33 milyon parametreye oldukça yakındır. Modelin toplam parametre sayısı bu konfigürasyon değerlerine göre bu şekilde hesaplanır.

In [7]:
import os
def print0(*args, **kwargs):
    # sadece master işlemden yazdıran değiştirilmiş print fonksiyonu
    # eğer bu dağıtılmış bir çalışma değilse, sadece normal print işlevi gibi çalışır
    if int(os.environ.get("RANK", 0)) == 0:
        print(*args, **kwargs)

In [8]:
import inspect

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd), # wte: kelime token gömmeleri
            wpe = nn.Embedding(config.block_size, config.n_embd), # wpe: pozisyon gömmeleri
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]), # h: transformer blokları
            ln_f = nn.LayerNorm(config.n_embd), # ln_f: çıktıdan önce son layer norm
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False) # lm_head: dil modelleme başlığı
        self.lm_head.LLMC_SKIP_INIT = 1 # bu kısmı başlatma, ağırlıkları bağlayacağız
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # tüm ağırlıkları başlat, çok dikkatli olmak için bir torch rng nesnesi kullan
        self.init_rng = torch.Generator()
        self.init_rng.manual_seed(42)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # GPT-2 makalesine göre rezidüel projeksiyonlara özel ölçeklenmiş başlatma uygula
            std = 0.02 if not hasattr(module, 'LLMC_RESIDUAL_SCALE_FLAG') else 0.02 / math.sqrt(2 * self.config.n_layer)
            # lm_head'i başlatmaktan kaçınmak istiyoruz, çünkü wte ile paylaşılan parametreler
            # ve wte Embedding başlatması sırasında zaten başlatıldı
            if not hasattr(module, 'LLMC_SKIP_INIT'):
                torch.nn.init.normal_(module.weight, mean=0.0, std=std, generator=self.init_rng)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02, generator=self.init_rng)

    def forward(self, idx, targets=None, return_logits=True):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Bu uzunluktaki bir diziyi ilerletemezsiniz: {t}, blok boyutu yalnızca {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # şekil (t)

        # GPT modelini ileri geçir
        tok_emb = self.transformer.wte(idx) # token gömmeleri şekli (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # pozisyon gömmeleri şekli (t, n_embd)
        x = tok_emb + pos_emb

        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # eğer istenen hedefler verilmişse, kaybı da hesapla
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # çıkarım zamanında mini optimizasyon: lm_head'i sadece son pozisyonda ileri geçir
            logits = self.lm_head(x[:, [-1], :]) # zaman boyutunu korumak için liste [-1] kullanılıyor
            loss = None

        # performans nedenlerinden dolayı, eğer gerekli değilse logits'i döndürmemek uygun olur
        if not return_logits:
            logits = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type, zero_stage):
        # aday parametrelerle başla
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # grad gerektirmeyenleri filtrele
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # optimizasyon grupları oluştur. 2D olan tüm parametreler weight decay'e tabi tutulur, diğerleri tutulmaz.
        # yani, matmul'lerdeki ve gömmelerdeki tüm ağırlık tensörleri decay edilir, tüm bias'lar ve layernorm'lar edilmez.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print0(f"decay edilen parametre tensörlerinin sayısı: {len(decay_params)}, {num_decay_params:,} parametre")
        print0(f"decay edilmeyen parametre tensörlerinin sayısı: {len(nodecay_params)}, {num_nodecay_params:,} parametre")
        # AdamW optimizasyonu oluştur ve mümkünse birleşik versiyonu kullan
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        print0(f"birleşik AdamW kullanılıyor: {use_fused}")
        if zero_stage == 1:
            print0("ZeroRedundancyOptimizer kullanılıyor")
            optimizer = ZeroRedundancyOptimizer(**optim_groups[0], optimizer_class=torch.optim.AdamW,
                                                lr=learning_rate, betas=betas, fused=use_fused)
            optimizer.add_param_group(optim_groups[1])
        else:
            print0("normal AdamW kullanılıyor")
            optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, fused=use_fused)
        return optimizer

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        idx (LongTensor şekli (b,t)) dizininin tamamlanmış halini al ve her seferinde tahminleri modele geri besle.
        Büyük olasılıkla model.eval() modunda çalışmak isteyeceksiniz.
        """
        for _ in range(max_new_tokens):
            # dizinin bağlamı çok uzarsa, blok boyutunda kesmemiz gerekir
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # dizideki indekse ait logits'leri almak için modeli ileri geçir
            logits, _ = self(idx_cond)
            # logits'leri son adımda alın ve istenen sıcaklıkla ölçeklendirin
            logits = logits[:, -1, :] / temperature
            # isteğe bağlı olarak logits'leri sadece top k seçenekleriyle kırpın
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # logits'leri (normalize edilmiş) olasılıklara dönüştürmek için softmax uygulayın
            probs = F.softmax(logits, dim=-1)
            # dağılımdan örnekleme yap
            idx_next = torch.multinomial(probs, num_samples=1)
            # örneklenen indeksi devam eden diziye ekle ve devam et
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [9]:
def _peek_data_shard(filename):
    """
    Dosyanın sadece başlık kısmını okur ve başlık verilerini döndürür.
    Burada başlık, dosyanın tamamını okuduğumuz ve byte olarak depoladığımız veri olarak ele alınmıştır.
    
    Args:
        filename (str): Okunacak dosyanın adı.

    Returns:
        int: Dosyadaki byte sayısı.
    """
    with open(filename, "rb") as f:
        tokens = f.read()  # Dosyayı byte olarak oku
        tokens = [x for x in tokens]  # Byte'ları listeye dönüştür
    return len(tokens)  # Byte'ların sayısını döndür

def _load_data_shard(filename):
    """
    Dosyayı okur ve içindeki tüm verileri byte olarak döndürür.
    
    Args:
        filename (str): Okunacak dosyanın adı.

    Returns:
        list: Dosyadaki byte'ların listesi.
    """
    with open(filename, "rb") as f:
        tokens = f.read()  # Dosyayı byte olarak oku
        tokens = [x for x in tokens]  # Byte'ları listeye dönüştür
    return tokens  # Byte'ların listesini döndür

In [10]:
import glob

class DistributedDataLoader:
    def __init__(self, filename_pattern, B, T, process_rank, num_processes):
        """
        Dağıtık veri yükleyici başlatılır.
        
        Args:
            filename_pattern (str): Yüklenmesi gereken dosyaların desenini belirten bir dize.
            B (int): Batch boyutu.
            T (int): Sequence uzunluğu.
            process_rank (int): İşlem sırası.
            num_processes (int): Toplam işlem sayısı.
        """
        self.process_rank = process_rank
        self.num_processes = num_processes
        self.B = B
        self.T = T

        # Desene uyan dosyaları bul ve sırala
        self.files = sorted(glob.glob(filename_pattern))
        assert len(self.files) > 0, f"Desene uyan hiçbir dosya bulunamadı: {filename_pattern}"

        # Tüm veri parçalarını yükle ve doğrula, toplam token sayısını say
        ntok_total = 0
        for fname in self.files:
            shard_ntok = _peek_data_shard(fname)
            assert shard_ntok >= num_processes * B * T + 1, f"Veri parçacığı {fname} mevcut ayar için çok küçük"
            ntok_total += shard_ntok
        self.ntok_total = ntok_total
        print0(f"Veri Yükleyici: Toplam token sayısı: {ntok_total:,} {len(self.files)} dosyada")

        # Başlatma işlemi
        self.current_shard = None
        self.reset()

    def reset(self):
        """
        Veri yükleyiciyi yeniden başlatır.
        """
        if self.current_shard != 0:
            self.current_shard = 0
            self.tokens = _load_data_shard(self.files[self.current_shard])
        self.current_position = self.process_rank * self.B * self.T

    def advance(self): 
        """
        Sonraki veri parçasına ilerler.
        """
        self.current_shard = (self.current_shard + 1) % len(self.files)
        self.current_position = self.process_rank * self.B * self.T
        self.tokens = _load_data_shard(self.files[self.current_shard])

    def next_batch(self):
        """
        Bir sonraki batch'i döndürür.
        
        Returns:
            x (torch.Tensor): Giriş tensoru.
            y (torch.Tensor): Hedef tensoru.
        """
        B = self.B
        T = self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        buf = torch.tensor(buf, dtype=torch.long)
        x = (buf[:-1]).view(B, T) # Girişler
        y = (buf[1:]).view(B, T) # Hedefler
        # Mevcut parçacıkta başlangıç işaretçisini ilerlet
        self.current_position += B * T * self.num_processes
        # Eğer bir sonraki batch'i yüklemek sınırları aşarsa, parçacığı ilerlet
        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            self.advance()
        return x, y

In [11]:
def write_fp16(tensor, file):
    """
    Writes a tensor to a file in float16 format.

    Parameters:
    - tensor: Tensor to be written.
    - file: File object to write to.
    """
    t = tensor.detach().cpu().to(torch.float16)  # Convert tensor to float16
    b = t.numpy().tobytes()  # Convert tensor to numpy array and then to bytes
    file.write(b)  # Write bytes to file

def write_fp32(tensor, file):
    """
    Writes a tensor to a file in float32 format.

    Parameters:
    - tensor: Tensor to be written.
    - file: File object to write to.
    """
    t = tensor.detach().cpu().to(torch.float32)  # Convert tensor to float32
    b = t.numpy().tobytes()  # Convert tensor to numpy array and then to bytes
    file.write(b)  # Write bytes to file

def write_bf16(tensor, file):
    """
    Writes a tensor to a file in bfloat16 format.

    Parameters:
    - tensor: Tensor to be written.
    - file: File object to write to.
    """
    t = tensor.detach().cpu().to(torch.bfloat16)  # Convert tensor to bfloat16
    t = t.view(torch.int16)  # Trick: reinterpret as int16 for numpy compatibility
    b = t.numpy().tobytes()  # Convert tensor to numpy array and then to bytes
    file.write(b)  # Write bytes to file

In [12]:
def write_tensors(model_tensors, L, file, dtype):
    """
    GPT-2 modelinin ağırlıklarını bir ikili dosyaya yazar.

    Parametreler:
    - model_tensors: Model ağırlıklarının tensorlerini içeren sözlük.
    - L: Transformer modelindeki katman sayısı.
    - file: Tensorlerin yazılacağı dosya nesnesi.
    - dtype: Tensorleri dönüştürmek için veri tipi ('float16', 'float32', 'bfloat16').

    Notlar:
    - 'dtype' parametresine bağlı olarak, uygun yazma fonksiyonunu ('write_fp16', 'write_fp32', 'write_bf16') seçer ve tensorleri dönüştürerek dosyaya yazar.
    - GPT-2 modelinin çeşitli bileşenleri için tensorler yazılır; bunlar arasında gömme katmanları, katman normalleştirme ağırlıkları, dikkat ağırlıkları ve sapmaları, MLP ağırlıkları ve sapmaları, ve son katman normalleştirme ağırlıkları bulunur.
    """
    assert dtype in {"float16", "float32", "bfloat16"}

    # 'dtype' parametresine göre uygun yazma fonksiyonunu seç
    write_fun = write_fp16 if dtype == "float16" else write_fp32 if dtype == "float32" else write_bf16

    # Model tensorlerini dosyaya yaz
    write_fun(model_tensors["transformer.wte.weight"], file)  # (V, C)
    write_fun(model_tensors["transformer.wpe.weight"], file)  # (T, C)

    for i in range(L):  # (L, C)
        write_fun(model_tensors[f"transformer.h.{i}.ln_1.weight"], file)

    for i in range(L):  # (L, C)
        write_fun(model_tensors[f"transformer.h.{i}.ln_1.bias"], file)

    for i in range(L):  # (L, 3C, C)
        write_fun(model_tensors[f"transformer.h.{i}.attn.c_attn.weight"], file)

    for i in range(L):  # (L, 3C)
        write_fun(model_tensors[f"transformer.h.{i}.attn.c_attn.bias"], file)

    for i in range(L):  # (L, C, C)
        write_fun(model_tensors[f"transformer.h.{i}.attn.c_proj.weight"], file)

    for i in range(L):  # (L, C)
        write_fun(model_tensors[f"transformer.h.{i}.attn.c_proj.bias"], file)

    for i in range(L):  # (L, C)
        write_fun(model_tensors[f"transformer.h.{i}.ln_2.weight"], file)

    for i in range(L):  # (L, C)
        write_fun(model_tensors[f"transformer.h.{i}.ln_2.bias"], file)

    for i in range(L):  # (L, 4C, C)
        write_fun(model_tensors[f"transformer.h.{i}.mlp.c_fc.weight"], file)

    for i in range(L):  # (L, 4C)
        write_fun(model_tensors[f"transformer.h.{i}.mlp.c_fc.bias"], file)

    for i in range(L):  # (L, C, 4C)
        write_fun(model_tensors[f"transformer.h.{i}.mlp.c_proj.weight"], file)

    for i in range(L):  # (L, C)
        write_fun(model_tensors[f"transformer.h.{i}.mlp.c_proj.bias"], file)

    write_fun(model_tensors["transformer.ln_f.weight"], file)  # (C, )
    write_fun(model_tensors["transformer.ln_f.bias"], file)  # (C, )

In [13]:
@torch.no_grad()
def pad_vocab(tensor, multiple=128, value=0):
    """
    Tensorin kelime dağarcığı boyutunu en yakın uygun katına kadar doldurur.

    Parametreler:
    - tensor: (V, C) şeklinde olan giriş tensorü, burada V orijinal kelime dağarcığı boyutunu ve C özellik sayısını belirtir.
    - multiple: Kelime dağarcığının doldurulacağı uygun kat.
    - value: Dolgu satırlarının doldurulacağı değer.

    Dönüş:
    - padded: Şekli (Vp, C) olan tensor, burada Vp doldurulmuş kelime dağarcığı boyutunu temsil eder.

    Notlar:
    - Bu fonksiyon tensorün kelime dağarcığı boyutunu (V) 'multiple' parametresi ile belirtilen en yakın katına kadar doldurur.
    - Bu doldurma işlemi, tensor işlemlerini özellikle GPU üzerinde daha verimli hale getirmek için yapılır.
    - Algoritmik olarak, bu işlem tensorün anlamını değiştirmez, yalnızca boyutlarını optimize eder.
    """
    assert tensor.ndim == 2  # Tensorün 2 boyutlu olduğundan emin ol (V, C)
    V, C = tensor.shape

    # Doldurulmuş kelime dağarcığı boyutunu 'multiple' ile belirtilen en yakın katına yuvarla
    Vp = ((V + multiple - 1) // multiple) * multiple

    # Gerekirse tensorü doldur
    pad_rows = Vp - V
    padded = tensor if pad_rows == 0 else F.pad(tensor, (0, 0, 0, pad_rows), value=value)

    assert padded.shape == (Vp, C)  # Doldurulmuş tensorün doğru şekilde olduğundan emin ol (Vp, C)

    return padded

In [14]:
def write_model(model, filename, dtype):
    """
    Modeli belirtilen dosyaya yazan fonksiyon.

    Parametreler:
    - model: Kaydedilecek model.
    - filename: Kaydedilecek dosyanın adı.
    - dtype: Tensorleri dönüştürmek için veri tipi ('float16', 'float32', 'bfloat16').

    Notlar:
    - 'dtype' parametresi, geçerli bir veri tipi olmalıdır ('float16', 'float32', 'bfloat16').
    - Belirtilen 'dtype' parametresine göre versiyon seçilir ve header oluşturulur.
    - Model parametreleri header'ın ardından dosyaya yazılır, önce kelime dağarcığı boyutu 'pad_vocab' fonksiyonuyla uygun bir katına kadar doldurulur.
    """
    assert dtype in {"float16", "float32", "bfloat16"}

    # Versiyon numarasını belirle
    version = {
        "float16": 2,  # 2: Tüm tensorler float16, doldurulmuş kelime dağarcığı
        "float32": 3,  # 3: Tüm tensorler float32, doldurulmuş kelime dağarcığı
        "bfloat16": 5,  # 5: Tüm tensorler bfloat16, doldurulmuş kelime dağarcığı
    }[dtype]

    # Header oluştur
    header = torch.zeros(256, dtype=torch.int32)
    header[0] = 20240326  # Magic sayı
    header[1] = version  # Kontrol noktası versiyonu
    header[2] = model.config.block_size
    header[3] = model.config.vocab_size
    header[4] = model.config.n_layer
    header[5] = model.config.n_head
    header[6] = model.config.n_embd

    # Parametreleri al
    params = {name: param.cpu() for name, param in model.named_parameters()}

    # Kelime dağarcığını 128'in katına kadar doldur
    wte = params["transformer.wte.weight"]  # (V, C)
    wte_padded = pad_vocab(wte)  # (Vp, C)
    params["transformer.wte.weight"] = wte_padded  # (Vp, C)
    print(f"{wte.size(0)} boyutundan {wte_padded.size(0)} boyutlu doldurulmuş kelime dağarcığı")
    header[7] = wte_padded.size(0)  # Header'da doldurulmuş kelime dağarcığı boyutunu sakla

    # Şimdi dosyaya yaz
    with open(filename, "wb") as file:
        file.write(header.numpy().tobytes())  # Header'ı yaz
        write_tensors(params, model.config.n_layer, file, dtype)  # Parametreleri yaz

    print(f"{filename} dosyası yazıldı")

In [15]:
def write_state(model, x, y, logits, loss, filename, dtype="float32"):
    """
    Model durumunu belirtilen dosyaya yazan fonksiyon.

    Parametreler:
    - model: Kaydedilecek model.
    - x: Modelin girdisi olan tensor.
    - y: Modelin hedef çıktısı olan tensor.
    - logits: Modelin çıktıları (logitler).
    - loss: Modelin kaybı (loss), tek bir float değeri.
    - filename: Kaydedilecek dosyanın adı.
    - dtype: Tensorleri dönüştürmek için veri tipi ('float32' varsayılan olarak).

    Notlar:
    - Bu fonksiyon, hata ayıklama için girdi, logitler, kayıp ve parametre gradyanlarını içeren bir durum bilgisi içerir.
    - 'dtype' parametresi, geçerli bir veri tipi olmalıdır ('float32', 'float16', 'bfloat16').
    - Header oluşturulur ve dosyaya yazılır.
    - Girdi (x), hedef (y), logitler, kayıp (loss) ve gradyanlar dosyaya yazılır.
    """
    header = torch.zeros(256, dtype=torch.int32)
    header[0] = 20240327  # Magic sayı
    header[1] = 2  # Çalışma durumu versiyonu = 2 (1 -> 2 doldurulmuş kelime dağarcığı değişiklikleri için)
    header[2] = x.size(0)  # Batch boyutu B
    header[3] = x.size(1)  # Batch'ın zamansal uzunluğu T

    # Parametre gradyanlarını al
    grads = {name: param.grad.cpu() for name, param in model.named_parameters()}

    # Kelime dağarcığını gradyanlarda da uygun katına kadar doldur
    wte_grad = grads["transformer.wte.weight"]  # (V, C)
    wte_grad_padded = pad_vocab(wte_grad, value=0)  # (Vp, C)
    grads["transformer.wte.weight"] = wte_grad_padded  # (Vp, C)
    print(f"Referans gradyanlarda doldurulmuş kelime dağarcığı boyutu {wte_grad.size(0)} boyutundan {wte_grad_padded.size(0)} boyutuna")

    # Dosyaya yaz
    with open(filename, "wb") as file:
        # Header
        file.write(header.numpy().tobytes())
        # Girdi x
        file.write(x.cpu().numpy().astype("int32").tobytes())  # (B, T)
        # Hedef y
        file.write(y.cpu().numpy().astype("int32").tobytes())  # (B, T)
        # Logitler (model ileri geçişinin sonucu)
        write_fp32(logits.cpu(), file)
        # Kayıp (çapraz entropi kaybının tek bir float değeri)
        write_fp32(loss.cpu(), file)
        # Gradyanlar
        write_tensors(grads, model.config.n_layer, file, dtype)

    print(f"{filename} dosyası yazıldı")

In [16]:
def write_tokenizer(enc, filename):
    """
    Tokenizer'ı belirtilen dosyaya yazan fonksiyon.

    Parametreler:
    - enc: Encoder (kodlayıcı) nesnesi.
    - filename: Kaydedilecek dosyanın adı.

    Notlar:
    - Bu fonksiyon, tokenizer'ın versiyonunu, token sayısını ve EOT (End-Of-Text) token'ını içeren bir başlık oluşturur ve dosyaya yazar.
    - Her bir token için uzunluğunu ve gerçek baytlarını bir dosyaya yazar.
    """
    n = enc.max_token_value + 1
    header = torch.zeros(256, dtype=torch.int32)
    header[0] = 20240328  # Magic sayı
    header[1] = 2  # Tokenizer versiyonu = 2 (1 -> 2: EOT token içerir)
    header[2] = n  # Token sayısı
    header[3] = enc.eot_token  # EOT (End-Of-Text) tokenı

    with open(filename, "wb") as file:
        file.write(header.numpy().tobytes())

        # Her bir token için işlem yap
        for i in range(n):
            b = enc.decode_bytes([i])  # Tokenı baytlara çöz
            length = len(b)
            assert length < 256, f"Token uzunluğu 255'i aşıyor: {length}"
            file.write(struct.pack("<B", length))  # Uzunluğu 1 byte olarak yaz (unsigned integer)
            file.write(b)  # Gerçek baytları yaz

    print(f"{filename} dosyası yazıldı")

In [17]:
# B, T = 2, 4  # Bu satır şu an yorum olarak işaretlenmiş durumda, kullanılmıyor gibi görünüyor.

""" 
GPTConfig sınıfıyla ilgili açıklamalar burada yer alabilir. Bu sınıf, GPT modeli için yapılandırma parametrelerini içerir.
"""
""" @dataclass
class GPTConfig:
    block_size: int = 128  # Blok boyutu
    vocab_size: int = 64   # Kelime dağarcığı boyutu
    n_layer: int = 4       # Katman sayısı
    n_head: int = 4        # Kafa (head) sayısı
    n_embd: int = 16       # Gömme boyutu
 """
B, T = 128, 32  # Batch boyutu ve zaman serisi uzunluğu tanımlanıyor.

# Cihaz belirleme
device = "cpu"  # Varsayılan olarak CPU kullanılıyor
if torch.cuda.is_available():  # Eğer CUDA kullanılabilirse,
    device = "cuda"  # CUDA cihazı kullanılıyor
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"  # CUDA Memory Pooling Subsystem (MPS) kullanılıyor
device_type = 'cuda' if 'cuda' in device else 'cpu'  # Cihaz tipi belirleme (cuda veya cpu)
print(f"Kullanılan cihaz: {device} ({device_type})")  # Kullanılan cihazı ve tipini ekrana yazdırma

Kullanılan cihaz: mps (cpu)


In [18]:
ddp_rank = 0  # DDP (DistributedDataParallel) kümesindeki sürecin sırası (rank)
ddp_local_rank = 0  # DDP kümesindeki yerel sıra (local rank)
zero_stage = 0  # ZeRO-3 optimizasyonunun aşaması
ddp_world_size = 1  # DDP kümesindeki toplam süreç sayısı (world size)
master_process = True  # Ana süreç (master process) kontrolü
seed_offset = 0  # Rastgelelik için başlangıç ofseti
total_batch_size = B * T  # Toplam batch boyutu (Batch boyutu * Zaman serisi uzunluğu)
tokens_per_fwdbwd = B * T  # İleri ve geri geçişte kullanılan toplam token sayısı
tokens_per_fwdbwd  # Bu değişken aynı zamanda tokens_per_fwdbwd olarak tanımlanmıştır

4096

In [19]:
# total_batch_size ve tokens_per_fwdbwd değişkenlerinin bölümünden kalanın sıfır olduğunu doğrulama
assert total_batch_size % tokens_per_fwdbwd == 0

# Gradient birikimi adımlarını hesaplamak için total_batch_size ve tokens_per_fwdbwd arasındaki bölümü gerçekleştiriyoruz
grad_accum_steps = total_batch_size // tokens_per_fwdbwd

# Hesaplanan gradient birikimi adımlarını ekrana yazdırma
print(f"Toplam istenen batch boyutu: {total_batch_size}")
print(f"=> Hesaplanan gradient birikimi adımları: {grad_accum_steps}")

Toplam istenen batch boyutu: 4096
=> Hesaplanan gradient birikimi adımları: 1


In [20]:
from contextlib import nullcontext  # nullcontext'ı içe aktar

# İstenilen dtype ve cihaz tipine göre bir bağlam yöneticisi kurma
# 'float16' için ptdtype belirleme
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16, 'float8': torch.float8_e4m3fn}['float16']
# Cihaz CUDA ise, otomatik karar verme (autocast) ile bağlam oluşturma
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype) if device_type == "cuda" else nullcontext()

# Rastgelelik ve çoğaltılabilirlik (reproducibility)
torch.manual_seed(42)  # CPU için rastgelelik tohumunu ayarlama
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)  # CUDA kullanılabilirse, CUDA için rastgelelik tohumunu ayarlama

In [21]:
# Yeni bir GPT modeli oluşturuyoruz ve yapılandırma nesnesi ile başlatıyoruz
model = GPT(GPTConfig())
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(32, 144)
    (wpe): Embedding(64, 144)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=144, out_features=432, bias=True)
          (c_proj): Linear(in_features=144, out_features=144, bias=True)
        )
        (ln_2): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=144, out_features=576, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=576, out_features=144, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=144, out_features=32, bias=False)
)

In [22]:
model.train()  # Modeli eğitim moduna geçiriyoruz. Bu, modelin eğitim sırasında gradyanları takip etmesini sağlar.

model.to(device)  # Modeli belirtilen cihaza taşıyoruz. 'device' değişkeni, önceden belirlenmiş bir cihaz türünü temsil eder (ör. 'cuda' veya 'cpu').

if False:  # Koşul her zaman yanlış olduğu için bu blok çalıştırılmaz.
    if hasattr(config, "coordinate_descent_tuning"):  # 'config' nesnesinin 'coordinate_descent_tuning' özelliği varsa,
        config.coordinate_descent_tuning = True  # Bu özelliği 'True' olarak ayarlıyoruz. Bu ayar, modelin koordinat iniş ayarlamalarını etkinleştirmek için kullanılabilir.
    print0("compiling the model...")  # "model derleniyor..." mesajını ekrana yazdırıyoruz. 'print0' işlevi muhtemelen 'print' işlevini çağırmak için kullanılan bir işlevdir.
    model = torch.compile(model)  # Modeli derlemek için 'torch.compile' işlevini kullanıyoruz. Ancak bu blok şu anda işletilmiyor.

In [23]:
# Eğitim için dağıtılmış veri yükleyicisi oluşturma
train_loader = DistributedDataLoader("egitim_jetonlari.bin", B, T, ddp_rank, ddp_world_size)
# "egitim_jetonlari.bin" dosyasından verileri yükler. B: batch boyutu, T: zaman serisi uzunluğu, ddp_rank: süreç sırası, ddp_world_size: toplam süreç sayısı.

# Doğrulama için dağıtılmış veri yükleyicisi oluşturma
val_loader = DistributedDataLoader("dogrulama_jetonlari.bin", B, T, ddp_rank, ddp_world_size)
# "dogrulama_jetonlari.bin" dosyasından verileri yükler. B: batch boyutu, T: zaman serisi uzunluğu, ddp_rank: süreç sırası, ddp_world_size: toplam süreç sayısı.

Veri Yükleyici: Toplam token sayısı: 1,520,928 1 dosyada
Veri Yükleyici: Toplam token sayısı: 168,991 1 dosyada


In [24]:
x, y = train_loader.next_batch()
# 'train_loader' üzerinden bir sonraki batch'i alır. Bu işlev, dağıtılmış veri yükleyicisine özgü olabilir ve bir sonraki veri batch'ini döndürür.

x, y = x.to(device), y.to(device)
# 'x' ve 'y' tensörlerini belirtilen 'device' (cihaz) üzerine taşır. Bu işlem genellikle GPU'lar arasında veri transferi için kullanılır.

x.shape, y.shape
# 'x' ve 'y' tensörlerinin şekillerini yazdırır. Bu, her bir tensörün boyutlarını ve şeklini kontrol etmek için kullanışlıdır.

(torch.Size([128, 32]), torch.Size([128, 32]))

In [25]:
logits, loss = model(x, y)
# Modeli, girdi ('x') ve etiket ('y') ile çağırarak çıktı tahminlerini ('logits') ve kaybı ('loss') hesaplar.

loss.backward()
# Kaybı kullanarak geri yayılım (backpropagation) işlemi yapar, yani gradyanları hesaplar ve modelin parametrelerine uygular.

# Model parametrelerini farklı veri tiplerinde ('float32' ve 'bfloat16') kaydetme işlemi
model_to_size = {"tek-harfli-gpt": "2.33M",  "gpt2": "124M", "gpt2-medium": "355M", "gpt2-large": "774M", "gpt2-xl": "1558M"}
model_to_size.update({f"d{d}": f"d{d}" for d in [12, 24, 36, 48]})
model_size_str = model_to_size["tek-harfli-gpt"]  # Örneğin "124M" veya "d12"
write_model(model, f"gpt2_{model_size_str}.bin", dtype="float16")  # 'float16' veri tipinde modeli kaydeder
write_model(model, f"gpt2_{model_size_str}_bf16.bin", dtype="bfloat16")  # 'bfloat16' veri tipinde modeli kaydeder

# Hata ayıklama için x, y, logits, loss ve gradyanları kaydetme işlemi
# Referans olarak her zaman 'fp32' (float32) veri tipinde saklanır (?)
write_state(model, x, y, logits, loss, f"gpt2_{model_size_str}_debug_state.bin")

# Optimizasyon için train_loader'ı sıfırlama
train_loader.reset()

# Model gradyanlarını açıkça sıfırlama işlemi
# Çünkü eğitim döngüsünde backward() ve ardından zero_grad() yaparız, bu da ilk eğitim adımında yanlış bir birikme yapabilir
model.zero_grad()

32 boyutundan 128 boyutlu doldurulmuş kelime dağarcığı
gpt2_2.33M.bin dosyası yazıldı
32 boyutundan 128 boyutlu doldurulmuş kelime dağarcığı
gpt2_2.33M_bf16.bin dosyası yazıldı
Referans gradyanlarda doldurulmuş kelime dağarcığı boyutu 32 boyutundan 128 boyutuna
gpt2_2.33M_debug_state.bin dosyası yazıldı


In [26]:
weight_decay = 0.0001
# Ağırlıkların güncellenmesi sırasında uygulanacak L2 düzenlileştirmesi miktarı.

learning_rate = 3e-5
# Optimizasyon algoritması tarafından kullanılacak başlangıç öğrenme oranı.

learning_rate_decay_frac = 0.0001
# Her iterasyonda öğrenme oranının azaltılacağı yüzde miktarı.

warmup_iters = 0
# Eğitim başlamadan önce uygulanacak "ısınma" adımı sayısı.

num_iterations = 1000
# Eğitimin toplam iterasyon sayısı.

val_loss_every = 0
# Doğrulama kaybının hesaplanacağı iterasyon sayısı. Sıfır ise doğrulama her iterasyonda hesaplanmaz.

val_max_steps = 20
# Doğrulama işlemi için maksimum adım sayısı.

sample_every = 0
# Örnekleme sıklığı. Modelin her kaç iterasyonda bir örnek üreteceği.

overfit_single_batch = 1
# Tek bir batch üzerinde aşırı uyumu kontrol etmek için kullanılan faktör.

inference_only = 0
# Yalnızca çıkarım modunda çalıştırılıp çalıştırılmayacağı. 1 ise yalnızca çıkarım yapılır, eğitim yapılmaz.

grad_clip = 1.0
# Gradyanların maksimum normu. Bu değeri aşıldığında gradyanlar belirtilen değere göre kırpılır.

In [27]:
ham_model = model # Modeli 'ham_model' adlı bir değişkene kopyalama.

In [28]:
optimizer = ham_model.configure_optimizers(
    weight_decay=weight_decay,  # Ağırlıkların güncellenmesi sırasında uygulanacak L2 düzenlileştirmesi miktarı.
    learning_rate=learning_rate,  # Optimizasyon algoritması tarafından kullanılacak öğrenme oranı.
    betas=(0.9, 0.95),  # Adam optimizer için beta parametreleri. İlk beta momentum terimi için, ikinci beta RMSProp'un exponential decay terimi için kullanılır.
    device_type=device,  # Kullanılan cihaz tipi ("cuda" veya "cpu").
    zero_stage=zero_stage  # Zamanında adımlama (ZeRO) aşamasını belirtir. Bellek kullanımını azaltmak için büyük model eğitimlerinde kullanılır.
)

decay edilen parametre tensörlerinin sayısı: 50, 2,999,808 parametre
decay edilmeyen parametre tensörlerinin sayısı: 98, 22,752 parametre
birleşik AdamW kullanılıyor: False
normal AdamW kullanılıyor


In [29]:
# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    min_lr = learning_rate * learning_rate_decay_frac
    # 1) lineer warmup için warmup_iters adımı
    if it < warmup_iters:
        return learning_rate * (it + 1) / warmup_iters
    # 2) eğer it > num_iterations ise min öğrenme oranını döndür
    if it > num_iterations:
        return min_lr
    # 3) ara değerlerde, min öğrenme oranına kadar kosinüs azalma kullan
    decay_ratio = (it - warmup_iters) / (num_iterations - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff 1'den başlar ve 0'a kadar iner
    return min_lr + coeff * (learning_rate - min_lr)

In [30]:
letter_list = ['a', 'b', 'c', 'ç', 'd', 'e', 'f', 'g', 'ğ', 'h', 'ı', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'ö', 'p', 'r', 's', 'ş', 't', 'u', 'ü', 'v', 'y', 'z', 
               ' ', '.', ',']

def encode(text):
    """
    Verilen metni harf listesindeki indislerle kodlar.

    Args:
    - text (str): Kodlanacak metin.

    Returns:
    - list: Metnin harf listesindeki indislerle kodlanmış hali.
    """
    return [letter_list.index(c) for c in text.lower()]

def decode(ids):
    """
    Verilen indis listesini orijinal metne dönüştürür.

    Args:
    - ids (list): Kodlanmış metnin indis listesi.

    Returns:
    - str: Kodlanmış indislerden orijinal metni oluşturan metin.
    """
    return ''.join([letter_list[i] for i in ids])

In [31]:
model.lm_head.weight[:5, :5]

tensor([[ 0.0385,  0.0297,  0.0180, -0.0421,  0.0136],
        [-0.0195,  0.0192,  0.0324,  0.0290,  0.0054],
        [ 0.0195, -0.0203, -0.0108, -0.0088, -0.0063],
        [-0.0099,  0.0227, -0.0092,  0.0284,  0.0170],
        [-0.0161, -0.0224,  0.0039, -0.0156, -0.0358]], device='mps:0',
       grad_fn=<SliceBackward0>)

In [32]:
encode(" ")

[29]

In [51]:
def calistir():
    # Örnek bir metin oluşturmak için bir örnek
    sample_text = " "

    # Metni harf listesi indisleriyle kodlayıp tensor olarak dönüştürme
    sample_tokens = encode(sample_text)
    sample_tokens = torch.tensor(sample_tokens, dtype=torch.long, device=device)[None, ...]

    # Modelin değerlendirme moduna geçmesi
    model.eval()

    # Gradyan hesaplama olmadan metin üretme işlemi yapma
    with torch.no_grad():
        sample_out = model.generate(sample_tokens, max_new_tokens=32, temperature=0.95, top_k=10)

    # Üretilen metni ekrana yazdırma
    print('---------------')
    print(f"Örnek giriş: {sample_text}")
    print(f"Üretilen çıktı: {decode(sample_out[0].tolist())}")
    print('---------------')

# Fonksiyonu çalıştırma
calistir()

---------------
Örnek giriş:  
Üretilen çıktı:  bu soruya verilen cevabı değerle
---------------


In [48]:
import time

timings = []
norm = -1.0   # Normalizasyon için varsayılan değer
for step in range(num_iterations + 1):
    t0 = time.time()  # Başlangıç zamanı

    last_step = (step == num_iterations)

    # Arada bir doğrulama veri setinde değerlendirme yapılır
    if (val_loss_every > 0 \
        and (step % val_loss_every == 0 or last_step)) \
        and (val_loader is not None):
        model.eval()
        val_loader.reset()
        with torch.no_grad():
            val_loss = 0.0
            for _ in range(val_max_steps):
                x, y = val_loader.next_batch()
                x, y = x.to(device), y.to(device)
                _, loss = model(x, y, return_logits=False)
                val_loss += loss.item()
            val_loss /= val_max_steps
        # Konsola ve dosyaya kaydetme
        print(f"val loss {val_loss}")

    # Arada bir modelin örnekleme yapılması, yalnızca ana süreçte gerçekleşir
    if (sample_every > 0 \
        and (step % sample_every == 0 or last_step)) \
        and master_process:
        model.eval()
        # Başlangıç için bir dizi oluşturulur, "" ile yeni bir dizinin başlangıcı belirtilir
        start_ids = [63]
        xg = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
        max_new_tokens = 32
        temperature = 1.0
        top_k = 40
        yg = ham_model.generate(xg, max_new_tokens, temperature=temperature, top_k=top_k)
        print('---------------')
        print(decode(yg[0].tolist()))
        print('---------------')

    # Modelin eğitim moduna geçmesi
    model.train()

    # Gradient birikimi yaparak istenen toplam veri boyutuna ulaşma
    lossf = 0.0  # Gradient birikimi aşamasında ortalama kaybı almak için
    for micro_step in range(grad_accum_steps):
        # Eğitim veri yükleyicisinden bir grup al
        if not overfit_single_batch \
            or (overfit_single_batch and step == 0 and micro_step == 0):
            x, y = train_loader.next_batch()
            x, y = x.to(device), y.to(device)

        # İleri geçiş
        with ctx:
            _, loss = model(x, y, return_logits=False)
            # Gradyan birikimini hesaba katmak için kaybı ölçeklendiriyoruz
            loss = loss / grad_accum_steps
            lossf += loss.detach()  # Ortalama kaybı takip etme
        # Geriye doğru geçiş
        if not inference_only:
            loss.backward()

    lossf = lossf.item()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    
    # Bu iterasyon için öğrenme oranını belirleme ve ayarlama
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    # Optimizer'ı güncelleme
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)
    
    # Eğitim bölümünün sonu
    # Buradan sonrası sadece tanısal amaçlı, yazdırma ve günlüklere ait
    # CPU'da tüm cihaz işlemlerinin tamamlanmasını bekleyin, böylece altındaki doğru iterasyon süreleri alabiliriz
    if device == "mps":
        torch.mps.synchronize()
    elif device == "cuda":
        torch.cuda.synchronize()
    
    # Zamanı ve çıktıları yazdırma
    t1 = time.time()
    tokens_per_second = grad_accum_steps * ddp_world_size * B * T / (t1-t0)
    print(f"step {step+1:4d}/{num_iterations} | train loss {lossf:.6f} | norm {norm:.4f} | lr {lr:.2e} | ({(t1-t0)*1000:.2f} ms | {tokens_per_second:.0f} tok/s)")

    # En son 20 iterasyonun zamanlarını takip etme
    if step > 0 and step > num_iterations - 20:
        timings.append(t1-t0)

step    1/1000 | train loss 4.126212 | norm 33.1827 | lr 3.00e-05 | (149.06 ms | 27479 tok/s)
step    2/1000 | train loss 3.906522 | norm 30.9736 | lr 3.00e-05 | (111.48 ms | 36741 tok/s)
step    3/1000 | train loss 3.666121 | norm 27.4738 | lr 3.00e-05 | (110.60 ms | 37033 tok/s)
step    4/1000 | train loss 3.471060 | norm 30.0103 | lr 3.00e-05 | (108.45 ms | 37770 tok/s)
step    5/1000 | train loss 3.287433 | norm 24.5010 | lr 3.00e-05 | (111.50 ms | 36737 tok/s)
step    6/1000 | train loss 3.106544 | norm 23.6764 | lr 3.00e-05 | (109.24 ms | 37497 tok/s)
step    7/1000 | train loss 2.944523 | norm 19.6703 | lr 3.00e-05 | (108.34 ms | 37806 tok/s)
step    8/1000 | train loss 2.803646 | norm 15.6169 | lr 3.00e-05 | (108.40 ms | 37785 tok/s)
step    9/1000 | train loss 2.682861 | norm 17.9463 | lr 3.00e-05 | (108.15 ms | 37875 tok/s)
step   10/1000 | train loss 2.581414 | norm 15.1700 | lr 3.00e-05 | (108.13 ms | 37881 tok/s)
step   11/1000 | train loss 2.486064 | norm 12.0744 | lr 3.0

In [49]:
""" 
Eğitim öncesi
tensor([[ 0.0385,  0.0297,  0.0180, -0.0421,  0.0136],
        [-0.0195,  0.0192,  0.0324,  0.0290,  0.0054],
        [ 0.0195, -0.0203, -0.0108, -0.0088, -0.0063],
        [-0.0099,  0.0227, -0.0092,  0.0284,  0.0170],
        [-0.0161, -0.0224,  0.0039, -0.0156, -0.0358]], device='mps:0',
       grad_fn=<SliceBackward0>)

Varmak istediğimiz yer
tensor([[ 0.0839,  0.0716,  0.0449, -0.0960,  0.0551],
        [-0.0607,  0.0496,  0.0738,  0.0693,  0.0313],
        [ 0.0485, -0.0739, -0.0587, -0.0393,  0.0301],
        [-0.0583,  0.0514, -0.0589,  0.0752,  0.0701],
        [-0.0523, -0.0578, -0.0065, -0.0553, -0.0861]], device='mps:0',
       grad_fn=<SliceBackward0>)
 """
model.lm_head.weight[:5, :5]



tensor([[ 0.0785,  0.0645,  0.0373, -0.0839,  0.0397],
        [-0.0535,  0.0411,  0.0644,  0.0563,  0.0212],
        [ 0.0361, -0.0643, -0.0469, -0.0099, -0.0125],
        [-0.0480,  0.0328, -0.0410,  0.0688,  0.0577],
        [-0.0423, -0.0519,  0.0101, -0.0371, -0.0678]], device='mps:0',
       grad_fn=<SliceBackward0>)

In [44]:
import numpy as np

# Son 20 zaman ölçümünün ortalamasını alarak yazdırma
timings = timings[-20:]
print(f"final {len(timings)} iters avg: {np.mean(timings)*1000:.3f}ms")

# Zirve bellek tüketimini yazdırma (CUDA için geçerlidir)
print(f"peak memory consumption: {torch.cuda.max_memory_allocated() // 1024 // 1024} MiB")

final 20 iters avg: 110.266ms
peak memory consumption: 0 MiB


In [54]:
calistir()

---------------
Örnek giriş:  
Üretilen çıktı:  yeni doğanlarımıza öğretiyoruz b
---------------
